# Importando librerias

In [13]:
import pandas as pd
import requests
from io import StringIO

# Limpiando
[Dataset](https://drive.google.com/file/d/15BMOwWdVQKsK8lBibax0_ks5K6HXxdFz/view?usp=sharing)

In [14]:
path = 'C:/Users/PC/Desktop/Proyectos Personales/Music/ny_phil.csv'
ny_phil = pd.read_csv(path)
ny_phil.head(2)

e:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Date,Location,Time,Venue,eventType,id,composerName,conductorName,interval,movement,workTitle,soloistInstrument,soloistName,soloistRoles,season,programID,orchestra
0,1842-12-07T05:00:00Z,"Manhattan, NY",8:00PM,Apollo Rooms,Subscription Season,38e072a7-8fc9-4f9a-8eac-3957905c0002,"Beethoven, Ludwig van","Hill, Ureli Corelli",NaN,NaN,"SYMPHONY NO. 5 IN C MINOR, OP.67",Soprano,"Otto, Antoinette",S,1842-43,3853.0,New York Philharmonic
1,1842-12-07T05:00:00Z,"Manhattan, NY",8:00PM,Apollo Rooms,Subscription Season,38e072a7-8fc9-4f9a-8eac-3957905c0002,"Beethoven, Ludwig van","Hill, Ureli Corelli",NaN,NaN,"SYMPHONY NO. 5 IN C MINOR, OP.67",Piano,"Scharfenberg, William",A,1842-43,3853.0,New York Philharmonic


In [15]:
#Dado que no nos es relevante la columna 'inverval', la eliminamos
ny_phil.drop('interval', axis = 1, inplace = True)
ny_phil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806179 entries, 0 to 806178
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Date               806179 non-null  object 
 1   Location           806179 non-null  object 
 2   Time               806179 non-null  object 
 3   Venue              806179 non-null  object 
 4   eventType          806179 non-null  object 
 5   id                 806179 non-null  object 
 6   composerName       737731 non-null  object 
 7   conductorName      570894 non-null  object 
 8   movement           340598 non-null  object 
 9   workTitle          737731 non-null  object 
 10  soloistInstrument  779437 non-null  object 
 11  soloistName        780096 non-null  object 
 12  soloistRoles       779834 non-null  object 
 13  season             781218 non-null  object 
 14  programID          781218 non-null  float64
 15  orchestra          781218 non-null  object 
dtypes:

In [16]:
#Los compositores y directores con NaN los llenamos con un 'Desconocido'
#Haremos lo mismo con los titulos de las obras y las orquestas con NaN
to_unknown = ['composerName', 'conductorName', 'workTitle', 'orchestra']
ny_phil[to_unknown] = ny_phil[to_unknown].fillna('Unknown')

#Si la obra consta de un solo movimiento se llenara como 0
ny_phil['movement'] = ny_phil['movement'].fillna('0')

to_none = ['soloistInstrument', 'soloistName', 'soloistRoles']
#Si no hay solita entonces lo llenamos con None
ny_phil[to_none] = ny_phil[to_none].fillna('None')

#Finalmente vamos a eliminar 
ny_phil.dropna(inplace = True)

ny_phil.head(2)

,Date,Location,Time,Venue,eventType,id,composerName,conductorName,movement,workTitle,soloistInstrument,soloistName,soloistRoles,season,programID,orchestra
0,1842-12-07T05:00:00Z,"Manhattan, NY",8:00PM,Apollo Rooms,Subscription Season,38e072a7-8fc9-4f9a-8eac-3957905c0002,"Beethoven, Ludwig van","Hill, Ureli Corelli",0,"SYMPHONY NO. 5 IN C MINOR, OP.67",Soprano,"Otto, Antoinette",S,1842-43,3853.0,New York Philharmonic
1,1842-12-07T05:00:00Z,"Manhattan, NY",8:00PM,Apollo Rooms,Subscription Season,38e072a7-8fc9-4f9a-8eac-3957905c0002,"Beethoven, Ludwig van","Hill, Ureli Corelli",0,"SYMPHONY NO. 5 IN C MINOR, OP.67",Piano,"Scharfenberg, William",A,1842-43,3853.0,New York Philharmonic


In [17]:
#Notamos que hay valores en la columna 'soloistInstrument' estan los valores ' ',
# 'Piano; Piano', 'Dancer;...', entonces vamos a eliminarlos 
ny_phil['soloistInstrument'].unique()

array(['Soprano', 'Piano', 'Violin', 'Viola', 'Cello', 'Contrabass',
       'Tenor', 'Flute', 'Clarinet', 'Bassoon', 'French Horn', 'None',
       'Oboe', 'Vocalist', 'Harp', 'Cornet', 'Alto', 'Bass',
       'Mens Chorus', 'Trumpet', 'Saxhorn', 'Womens Chorus', 'Baritone',
       'Chorus', 'Mezzo-Soprano', 'Organ', 'Trombone', 'Contralto',
       'Speaker', 'SATB Chorus', 'Narrator', 'Boys Choir', 'Reader',
       'Bass Clarinet', 'Ensemble', 'Other', 'Orchestra', 'Vocal Group',
       ' ', 'Elementary Chorus', 'Advanced Chorus', 'Piccolo',
       'Sing-Along', "Viola d'amore", 'English Horn', 'Harpsichord',
       'Dancer', 'Clavier', 'Viola tenore', 'Reciter',
       'Unspecified Voice', "Children's Chorus", 'Vocal Quartet',
       'Harmonium', 'String Quartet', 'Solo Voice', 'Duo-Art Piano',
       'Trombone Choir', 'Actor', 'Drum', 'Xylophone', 'Folk Singer',
       'Conductor', 'Treble [Boy soprano]', 'Tuba', 'Piano trio',
       'E-Flat Clarinet', 'Timpani', 'Percussion',
       

In [18]:
errors = [' ', 'Piano; Piano', 'Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Choreographer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Scenic Design',
       'Dancer; Choreographer',
       'Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer',
       'Dancer; Dancer; Choreographer; Choreographer',
       'Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Dancer; Choreographer; Dancer; Dancer']

ny_phil = ny_phil[~ny_phil['soloistInstrument'].isin(errors)]

## Transformamos las columnas

In [19]:
#Separamos el Estado de Location
state = ny_phil['Location'].str.split(', ', expand = True)
ny_phil['Location'] = state[0]
ny_phil['State'] = state[1]
ny_phil.head(2)

,Date,Location,Time,Venue,eventType,id,composerName,conductorName,movement,workTitle,soloistInstrument,soloistName,soloistRoles,season,programID,orchestra,State
0,1842-12-07T05:00:00Z,Manhattan,8:00PM,Apollo Rooms,Subscription Season,38e072a7-8fc9-4f9a-8eac-3957905c0002,"Beethoven, Ludwig van","Hill, Ureli Corelli",0,"SYMPHONY NO. 5 IN C MINOR, OP.67",Soprano,"Otto, Antoinette",S,1842-43,3853.0,New York Philharmonic,NY
1,1842-12-07T05:00:00Z,Manhattan,8:00PM,Apollo Rooms,Subscription Season,38e072a7-8fc9-4f9a-8eac-3957905c0002,"Beethoven, Ludwig van","Hill, Ureli Corelli",0,"SYMPHONY NO. 5 IN C MINOR, OP.67",Piano,"Scharfenberg, William",A,1842-43,3853.0,New York Philharmonic,NY


In [20]:
ny_phil['Date'] =  ny_phil['Date'].str.split('T', expand = True)[0]
ny_phil.head(2)

,Date,Location,Time,Venue,eventType,id,composerName,conductorName,movement,workTitle,soloistInstrument,soloistName,soloistRoles,season,programID,orchestra,State
0,1842-12-07,Manhattan,8:00PM,Apollo Rooms,Subscription Season,38e072a7-8fc9-4f9a-8eac-3957905c0002,"Beethoven, Ludwig van","Hill, Ureli Corelli",0,"SYMPHONY NO. 5 IN C MINOR, OP.67",Soprano,"Otto, Antoinette",S,1842-43,3853.0,New York Philharmonic,NY
1,1842-12-07,Manhattan,8:00PM,Apollo Rooms,Subscription Season,38e072a7-8fc9-4f9a-8eac-3957905c0002,"Beethoven, Ludwig van","Hill, Ureli Corelli",0,"SYMPHONY NO. 5 IN C MINOR, OP.67",Piano,"Scharfenberg, William",A,1842-43,3853.0,New York Philharmonic,NY


In [21]:
ny_phil.columns = ny_phil.columns.str.capitalize()
ny_phil.head(2)

,Date,Location,Time,Venue,Eventtype,Id,Composername,Conductorname,Movement,Worktitle,Soloistinstrument,Soloistname,Soloistroles,Season,Programid,Orchestra,State
0,1842-12-07,Manhattan,8:00PM,Apollo Rooms,Subscription Season,38e072a7-8fc9-4f9a-8eac-3957905c0002,"Beethoven, Ludwig van","Hill, Ureli Corelli",0,"SYMPHONY NO. 5 IN C MINOR, OP.67",Soprano,"Otto, Antoinette",S,1842-43,3853.0,New York Philharmonic,NY
1,1842-12-07,Manhattan,8:00PM,Apollo Rooms,Subscription Season,38e072a7-8fc9-4f9a-8eac-3957905c0002,"Beethoven, Ludwig van","Hill, Ureli Corelli",0,"SYMPHONY NO. 5 IN C MINOR, OP.67",Piano,"Scharfenberg, William",A,1842-43,3853.0,New York Philharmonic,NY


In [22]:
#Pasamos a datetime
categories = ['Location', 'Venue', 'Eventtype', 'Composername', 'Conductorname', 'Soloistinstrument', 'Soloistname', 'Season', 'Orchestra']
ny_phil[categories] = ny_phil[categories].astype('category') 

#Indicamos el indice
ny_phil.index = ny_phil['Date']
ny_phil.drop('Date', axis = 1, inplace = True)
ny_phil.head(2)

,Location,Time,Venue,Eventtype,Id,Composername,Conductorname,Movement,Worktitle,Soloistinstrument,Soloistname,Soloistroles,Season,Programid,Orchestra,State
Date,,,,,,,,,,,,,,,,
1842-12-07,Manhattan,8:00PM,Apollo Rooms,Subscription Season,38e072a7-8fc9-4f9a-8eac-3957905c0002,"Beethoven, Ludwig van","Hill, Ureli Corelli",0,"SYMPHONY NO. 5 IN C MINOR, OP.67",Soprano,"Otto, Antoinette",S,1842-43,3853.0,New York Philharmonic,NY
1842-12-07,Manhattan,8:00PM,Apollo Rooms,Subscription Season,38e072a7-8fc9-4f9a-8eac-3957905c0002,"Beethoven, Ludwig van","Hill, Ureli Corelli",0,"SYMPHONY NO. 5 IN C MINOR, OP.67",Piano,"Scharfenberg, William",A,1842-43,3853.0,New York Philharmonic,NY


In [23]:
ny_phil[ny_phil['Composername'].str.contains('Mahler')]

,Location,Time,Venue,Eventtype,Id,Composername,Conductorname,Movement,Worktitle,Soloistinstrument,Soloistname,Soloistroles,Season,Programid,Orchestra,State
Date,,,,,,,,,,,,,,,,
1904-11-06,Manhattan,3:00PM,Carnegie Hall,Subscription Season,42fcbb70-8121-4c81-892a-ba14e1bf73e7,"Mahler, Gustav","Damrosch, Walter",0,SYMPHONY NO. 4 IN G MAJOR,Soprano,"De Montjau, Etta",S,1904-05,8149.0,New York Symphony,NY
1904-11-06,Manhattan,3:00PM,Carnegie Hall,Subscription Season,42fcbb70-8121-4c81-892a-ba14e1bf73e7,"Mahler, Gustav","Damrosch, Walter",0,SYMPHONY NO. 4 IN G MAJOR,Soprano,"De Montjau, Etta",S,1904-05,8149.0,New York Symphony,NY
1904-11-06,Manhattan,3:00PM,Carnegie Hall,Subscription Season,42fcbb70-8121-4c81-892a-ba14e1bf73e7,"Mahler, Gustav","Damrosch, Walter",0,SYMPHONY NO. 4 IN G MAJOR,Soprano,"De Montjau, Etta",S,1904-05,8149.0,New York Symphony,NY
1904-11-06,Manhattan,3:00PM,Carnegie Hall,Subscription Season,42fcbb70-8121-4c81-892a-ba14e1bf73e7,"Mahler, Gustav","Damrosch, Walter",0,SYMPHONY NO. 4 IN G MAJOR,Soprano,"De Montjau, Etta",S,1904-05,8149.0,New York Symphony,NY
1904-11-06,Manhattan,3:00PM,Carnegie Hall,Subscription Season,42fcbb70-8121-4c81-892a-ba14e1bf73e7,"Mahler, Gustav","Damrosch, Walter",0,SYMPHONY NO. 4 IN G MAJOR,Soprano,"De Montjau, Etta",S,1904-05,8149.0,New York Symphony,NY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-06-09,Manhattan,8:00PM,David Geffen Hall,Subscription Season,2346a603-9629-4c29-9a2c-023367cbe493,"Mahler, Gustav","Gilbert, Alan",0,SYMPHONY NO. 7 IN E MINOR (SONG OF THE NIGHT),Jazz Bass,"Henriquez, Carlos",A,2016-17,14180.0,New York Philharmonic,NY
2017-06-09,Manhattan,8:00PM,David Geffen Hall,Subscription Season,2346a603-9629-4c29-9a2c-023367cbe493,"Mahler, Gustav","Gilbert, Alan",0,SYMPHONY NO. 7 IN E MINOR (SONG OF THE NIGHT),Trumpet,"Marsalis, Wynton",S,2016-17,14180.0,New York Philharmonic,NY
2017-06-09,Manhattan,8:00PM,David Geffen Hall,Subscription Season,2346a603-9629-4c29-9a2c-023367cbe493,"Mahler, Gustav","Gilbert, Alan",0,SYMPHONY NO. 7 IN E MINOR (SONG OF THE NIGHT),Piano,"Nimmer, Daniel",A,2016-17,14180.0,New York Philharmonic,NY


In [24]:
# concerts.loc['2010':]